<a href="https://colab.research.google.com/github/MananSuri27/MedVidClassification/blob/main/MedVidCL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MedVid QnA

In [1]:
!pip install ktrain

     |████████████████████████████████| 25.3 MB 44.9 MB/s 
     |████████████████████████████████| 22.3 MB 1.4 MB/s 
     |████████████████████████████████| 981 kB 35.6 MB/s 
     |████████████████████████████████| 263 kB 44.1 MB/s 
     |████████████████████████████████| 2.8 MB 30.9 MB/s 
     |████████████████████████████████| 1.2 MB 42.8 MB/s 
     |████████████████████████████████| 468 kB 45.0 MB/s 
     |████████████████████████████████| 895 kB 38.7 MB/s 
     |████████████████████████████████| 596 kB 38.3 MB/s 
     |████████████████████████████████| 67 kB 720 kB/s 
     |████████████████████████████████| 3.3 MB 34.6 MB/s 
  Created wheel for ktrain: filename=ktrain-0.29.3-py3-none-any.whl size=25295410 sha256=a52d4db536c3ea1fe0aff2f53711c473cfaa340eca2f6c9d668336e0ca10b5b1
  Stored in directory: /root/.cache/pip/wheels/04/d5/2a/1a1826e16685841c68a43c31c8a892452b105f9467f64e49e6
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9931 sha256=38b4cc0c9680ab7

In [2]:
import numpy as np
import pandas as pd
import json
import ktrain


In [17]:
trainF = open('train.json')
trainjson = json.load(trainF)

valF = open('val.json')
valjson = json.load(valF)

testF = open('test.json')
testjson = json.load(testF)


In [18]:
for row in trainjson:
  l = len(row['video_sub_title'])
  row['text'] = row['video_title'] +" "+ row['video_sub_title'][round(l/4):l]

for row in valjson:
  l = len(row['video_sub_title'])
  row['text'] = row['video_title'] +" "+ row['video_sub_title'][round(l/4):l]

for row in testjson:
  l = len(row['video_sub_title'])
  row['text'] = row['video_title'] +" "+ row['video_sub_title'][round(l/4):l]


In [19]:
train = pd.json_normalize(trainjson)
val = pd.json_normalize(valjson)
test = pd.json_normalize(testjson)



In [6]:
print(train)

         video_id                                   video_link  \
0     dnzpr6Yj29I  https://www.youtube.com/watch?v=dnzpr6Yj29I   
1     nFc3ppL5faU  https://www.youtube.com/watch?v=nFc3ppL5faU   
2     KmpHXlAR_R4  https://www.youtube.com/watch?v=KmpHXlAR_R4   
3     7tsT8iFcBqE  https://www.youtube.com/watch?v=7tsT8iFcBqE   
4     rMFDVLJKzCQ  https://www.youtube.com/watch?v=rMFDVLJKzCQ   
...           ...                                          ...   
4212  -9lL22o4DZg  https://www.youtube.com/watch?v=-9lL22o4DZg   
4213  5KNWWUX38W0  https://www.youtube.com/watch?v=5KNWWUX38W0   
4214  cS4Nz6FuEjw  https://www.youtube.com/watch?v=cS4Nz6FuEjw   
4215  M34-NDKrik0  https://www.youtube.com/watch?v=M34-NDKrik0   
4216  mv94XXKfGN4  https://www.youtube.com/watch?v=mv94XXKfGN4   

                                            video_title  \
0                                   Cat Scratch Disease   
1                              How to Increase Estrogen   
2     How to reduce Elevated C

In [7]:
max_len = 512
batch_size = 6
learning_rate = 2e-5
epochs = 2

In [8]:
# Transformer Model

model_='emilyalsentzer/Bio_ClinicalBERT'

from ktrain import text
t_mod = text.Transformer(model_, maxlen=max_len, classes = [0,1])


'''Converting split data to list [so it can processed]'''
#train
X_tr = train['text'].tolist()
y_tr = train['label'].tolist()

#test
X_ts = val['text'].tolist()
y_ts = val['label'].tolist()


# Pre-processing training & test data
train = t_mod.preprocess_train(X_tr,y_tr)
test = t_mod.preprocess_train(X_ts,y_ts)

# Model Classifier
model = t_mod.get_classifier()

learner = ktrain.get_learner(model, train_data=train, val_data=test, batch_size=batch_size)

/usr/local/lib/python3.7/dist-packages/ktrain/text/preprocessor.py:422: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')


Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

preprocessing train...
language: en
train sequence lengths:
	mean : 627
	95percentile : 1676
	99percentile : 2175


/usr/local/lib/python3.7/dist-packages/ktrain/utils.py:621: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing train...
language: en
train sequence lengths:
	mean : 609
	95percentile : 1554
	99percentile : 2122


404 Client Error: Not Found for url: https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT/resolve/main/tf_model.h5


Is Multi-Label? False


/usr/local/lib/python3.7/dist-packages/ktrain/text/preprocessor.py:1069: UserWarning: Could not find Tensorflow version of model.  Attempting to download/load PyTorch version as TensorFlow model using from_pt=True. You will need PyTorch installed for this.
  warnings.warn('Could not find Tensorflow version of model.  Attempting to download/load PyTorch version as TensorFlow model using from_pt=True. ' +\


Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [9]:
learner.fit_onecycle(lr=learning_rate,epochs=3)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
703/703 [==============================] - 1066s 1s/step - loss: 0.5510 - accuracy: 0.7814 - val_loss: 0.3175 - val_accuracy: 0.8633
Epoch 2/3
703/703 [==============================] - 1038s 1s/step - loss: 0.3112 - accuracy: 0.8907 - val_loss: 0.2362 - val_accuracy: 0.9233
Epoch 3/3
703/703 [==============================] - 1037s 1s/step - loss: 0.1881 - accuracy: 0.9322 - val_loss: 0.2373 - val_accuracy: 0.9033


In [10]:
# from ktrain import text as text

# (x_train, y_train), (x_test, y_test), preproc = text.texts_from_df(train_df=train,
#                                                                    text_column = 'text',
#                                                                    label_columns = 'label',
#                                                                    val_df = val,
#                                                                    maxlen = 256,
#                                                                    preprocess_mode = 'bert')

In [11]:
# model = text.text_classifier(name='bert',
#                              train_data=(x_train, y_train),
#                              preproc=preproc)

In [12]:
# learner = ktrain.get_learner(model=model,
#                              train_data=(x_train, y_train),
#                              batch_size=16)

In [13]:
# learner.fit_onecycle(lr=2e-5,epochs=1)

In [14]:
predictor = ktrain.get_predictor(learner.model, t_mod)

In [20]:
y_predict = predictor.predict(np.asarray(test['text']))

In [21]:
print(y_predict)

['Medical Non-instructional', 'Medical Non-instructional', 'Medical Non-instructional', 'Medical Non-instructional', 'Non-medical', 'Medical Instructional', 'Non-medical', 'Medical Non-instructional', 'Medical Instructional', 'Medical Instructional', 'Medical Non-instructional', 'Medical Non-instructional', 'Medical Non-instructional', 'Medical Non-instructional', 'Medical Instructional', 'Non-medical', 'Medical Non-instructional', 'Medical Instructional', 'Medical Non-instructional', 'Medical Non-instructional', 'Non-medical', 'Non-medical', 'Medical Non-instructional', 'Medical Non-instructional', 'Medical Non-instructional', 'Medical Instructional', 'Non-medical', 'Medical Non-instructional', 'Medical Non-instructional', 'Medical Non-instructional', 'Medical Non-instructional', 'Medical Instructional', 'Non-medical', 'Non-medical', 'Medical Non-instructional', 'Medical Instructional', 'Medical Instructional', 'Medical Instructional', 'Medical Non-instructional', 'Medical Non-instruc

In [31]:
ans  = []

for i in range(len(test)):
  newRow = {"video_id":testjson[i]["video_id"] , "label":y_predict[i]}
  ans.append(newRow)




In [32]:

with open("predictions.json", "w") as final:
  json.dump(ans, final)

In [24]:
y_val = predictor.predict(np.asarray(val['text']))

In [27]:
print(len(val), len(y_test))

300 300


In [33]:
ans  = []

for i in range(len(val)):
  newRow = {"video_id":valjson[i]["video_id"] , "label":y_val[i]}
  ans.append(newRow)

In [34]:


with open("solution.json", "w") as final:
  json.dump(ans, final)


In [22]:
y_test = np.asarray(test["label"])

In [30]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
y_test= np.asarray(val["label"])
cm = confusion_matrix(y_test, y_val)
print(cm)
f1 = f1_score(y_test, y_val, average = "macro" )
print(f1)
accuracy_score(y_test, y_val)

[[79 20  1]
 [ 2 97  1]
 [ 3  2 95]]
0.9030024687028342


0.9033333333333333